In [1]:
import tensorflow as tf
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
def get_idx2word(_index_from=3):
    word2idx = tf.keras.datasets.imdb.get_word_index()
    word2idx = {k:(v+_index_from) for k,v in word2idx.items()}
    word2idx["<pad>"] = 0
    word2idx["<start>"] = 1
    word2idx["<unk>"] = 2
    idx2word = {idx: w for w, idx in word2idx.items()}
    return idx2word

In [3]:
idx2word = get_idx2word()
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=20000)

sess = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
def make_df(x, y):
    return sess.createDataFrame(
        [(int(y_), [idx2word[idx] for idx in x_]) for x_, y_ in zip(x, y)],
        ['label', 'words'])

In [8]:
pipeline = Pipeline(stages=[
    CountVectorizer(inputCol='words',
                    outputCol='tf'),
    IDF(inputCol='tf',
        outputCol='tfidf'),
    NaiveBayes(smoothing=1.0, 
               modelType='multinomial', 
              featuresCol='tfidf'),
])

In [6]:
df_train = make_df(X_train, y_train)
df_test = make_df(X_test, y_test)

In [9]:
prediction = pipeline.fit(df_train).transform(df_test)
print("Testing Accuracy: %.3f" % MulticlassClassificationEvaluator().evaluate(prediction))

Testing Accuracy: 0.801
